## Model Testing

Now that we have a model for classifying sounds, lets apply it to classify sounds from a microphone. The tensorflow retraining example has a script for labelling images.

We modify this script to label sounds from the microphone. First, the script streams audio from the mic using pyaudio, and uses the webrtcvad package to detect if sound is present at the microphone. If a sound is present it is recorded for 3 seconds, and then converted into a spectrogram and finally labelled.

In [1]:
!pip install webrtcvad

In [2]:
# Import Dependencies
import webrtcvad
import pyaudio
import wave
import os, sys
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import tensorflow as tf

In [3]:
vad = webrtcvad.Vad()
vad.set_mode(2)
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 32000
CHUNK = 960
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "file.wav"

In [4]:
audio = pyaudio.PyAudio()

# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)

In [5]:
# Unpersists graph from file
with tf.gfile.FastGFile("./retrained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

Instructions for updating:
Use tf.gfile.GFile.


In [6]:
with tf.Session() as sess:
    # Feed the image_data as input to the graph and get first prediction
    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

    while True:

        frames = []
        frameCount = 0

        while frameCount < 5:
            data = stream.read(CHUNK, exception_on_overflow=False)

            if vad.is_speech(data, RATE):
                frameCount+=1;
            else:
                frameCount = 0;
            # print frameCount

        print("recording...")
        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)
        print("finished recording")

        waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        waveFile.setnchannels(CHANNELS)
        waveFile.setsampwidth(audio.get_sample_size(FORMAT))
        waveFile.setframerate(RATE)
        waveFile.writeframes(b''.join(frames))
        waveFile.close()

        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

        # change this as you see fit
        audio_path = 'file.wav'
        image_path = 'tmp/tmp.jpg'

        y, sr = librosa.load(audio_path)

        # Let's make and display a mel-scaled power (energy-squared) spectrogram
        S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)

        # Convert to log scale (dB). We'll use the peak power as reference.
        log_S = librosa.core.amplitude_to_db(S, ref=np.max)

        # Make a new figure
        fig = plt.figure(figsize=(12,4))
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)

        # Display the spectrogram on a mel scale
        # sample rate and hop length parameters are used to render the time axis
        librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')

        # Make the figure layout compact

        #plt.show()
        plt.savefig(image_path)
        plt.close()

        # Read in the image_data
        image_data = tf.gfile.FastGFile(image_path, 'rb').read()

        # Loads label file, strips off carriage return
        label_lines = [line.rstrip() for line
                           in tf.gfile.GFile("./retrained_labels.txt")]

        predictions = sess.run(softmax_tensor, \
                 {'DecodeJpeg/contents:0': image_data})

        # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]

        print('%s (score = %.5f)' % (label_lines[top_k[0]], predictions[0][top_k[0]]))
        print("")

# stop Recording
stream.stop_stream()
stream.close()
audio.terminate()

recording...
finished recording
dog bark (score = 0.75024)

recording...
finished recording
dog bark (score = 0.34954)

recording...
finished recording
drilling (score = 0.42318)

recording...
finished recording
air conditioner (score = 0.29856)

recording...
finished recording
dog bark (score = 0.50650)

recording...
finished recording
dog bark (score = 0.36311)

recording...
finished recording
air conditioner (score = 0.26999)

recording...
finished recording
dog bark (score = 0.42728)

recording...
finished recording
dog bark (score = 0.41173)

recording...
finished recording
dog bark (score = 0.63066)

recording...
finished recording
drilling (score = 0.29734)

recording...
finished recording
dog bark (score = 0.40172)

recording...
finished recording
dog bark (score = 0.29440)

recording...
finished recording
dog bark (score = 0.73827)

recording...
finished recording
dog bark (score = 0.38500)

recording...
finished recording
dog bark (score = 0.47050)

recording...
finished reco

KeyboardInterrupt: 